In [1]:
import numpy as np

In [2]:
X = 1500
Y = 1500
Z = 20
K = 3
data = np.random.rand(X,Y,Z).astype('float32')

# Try 1

In [3]:

# def vectors_comparison(vector1, vector2):
#     res = np.multiply(vector1, vector2)
#     res = np.sum(np.power(res, 2))
#     return res

# %%prun -l 20
# comp_res = {}
# for x in range(X): 
#     comp_res[x] = {}
    
#     for y in range(Y):
#         comp_res[x][y] = {}
#         col = data[x, y, :]
        
#         for x_iter in range(max(0, x-K), min(X, x+K+1)):  #  for x_iter in range(x, min(X, x+K+1)): 
#             comp_res[x][y][x_iter] = {}
            
#             for y_iter in range(max(0, y-K), min(Y, y+K+1)):

#                 if x_iter in comp_res: # придумать попроще
#                     if y_iter in comp_res[x_iter]:
#                         if x in comp_res[x_iter][y_iter]:
#                             if y in comp_res[x_iter][y_iter][x]:
#                                 comp_res[x][y][x_iter][y_iter] = comp_res[x_iter][y_iter][x][y]
#                                 continue

#                 if (x_iter != x) or (y_iter != y): # подумать над форматом кэширования
#                     comp_res[x][y][x_iter][y_iter] = vectors_comparison(col, data[x_iter, y_iter])

# neighbours_comp = np.zeros((X,Y)).reshape(X,Y)
# for x in range(X):
#     for y in range(Y):
#         tmp_arr = []
#         for x_iter in comp_res[x][y]:
#             for y_iter in comp_res[x][y][x_iter]:
#                 tmp_arr.append(comp_res[x][y][x_iter][y_iter])
#         neighbours_comp[x][y] = np.mean(tmp_arr)


# Try 2

In [4]:
# %%time


# res = np.zeros(data.shape[0:2])
# for x in range(X):
#     for y in range(Y):        
#         col = data[x, y, :]        
#         for x_idx in range(-K, K+1):
#             for y_idx in range(-K, K+1):
#                 if (0 <= x + x_idx < X) and (0 <= y + y_idx < Y):
#                     res[x, y] += vectors_comparison(col, data[x + x_idx, y + y_idx])
#         res[x, y] /= (min(x + x_idx, X - 1) - max(0, x - x_idx) + 1) * (min(y + y_idx, Y - 1) - max(0, y - y_idx) + 1) - 1 


# Try 3: numba

In [5]:
import numba
from numba import njit

In [6]:
X = np.int32(1500)
Y = np.int32(1500)
Z = np.int32(20)
K = np.int32(3)
data = np.random.rand(X,Y,Z).astype('float32')

In [7]:
%%time

@njit(numba.types.float32(numba.types.float32[:], 
                          numba.types.float32[:]),
      fastmath=True)
def vectors_comparison(vector1, vector2):
    res = np.multiply(vector1, vector2)
    res = np.sum(np.power(res, 2))
    return res

@njit(numba.types.float32[:, :](numba.types.float32[:, :, :], 
                                numba.types.int32, 
                                numba.types.int32, 
                                numba.types.int32, 
                                numba.types.int32),
    locals={'res': numba.types.float32[:, :],
            'col': numba.types.float32[:]}, parallel=True)
def get_comparison(data, X, Y, Z, K):
    res = np.zeros(data.shape[0:2], dtype='float32')
    for x in range(X):
        for y in range(Y):        
            col = data[x, y, :]        
            sss = np.float32(0.0)
            for x_idx in range(-K, K+1):
                for y_idx in range(-K, K+1):
                    if (0 <= x + x_idx < X) and (0 <= y + y_idx < Y):
                        sss += vectors_comparison(col, data[x + x_idx, y + y_idx])
            #count = (min(x + x_idx, X - 1) - max(0, x - x_idx) + 1) * (min(y + y_idx, Y - 1) - max(0, y - y_idx) + 1) - 1
            res[x, y] = sss #/ count
    return res

CPU times: user 1.86 s, sys: 5.84 s, total: 7.7 s
Wall time: 1.14 s


In [8]:
%%time
res = get_comparison(data, X, Y, Z, K)

CPU times: user 16.2 s, sys: 194 ms, total: 16.4 s
Wall time: 16.3 s
